In [1]:
# used pretrained model to perform inference



In [2]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator

model_path = "/home/saurabh/Documents/tf_orange/tf_orange/models/model.ckpt"

In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
# we also need a fixed vocabulary 
import re

word_dictionary = {}

with open("/home/saurabh/Documents/tf_orange/tf_orange/words/words.txt") as file:
    for i , line in enumerate(file):
        
        line = line.replace("\n", "")
        word_dictionary[line] = i


In [5]:
# Training Parameters
learning_rate = 0.001
training_steps = 5
batch_size = 1
display_step = 200
num_features = 13


num_classes = 9 # words.txt , all words plus space word  ( 10001) + CTC symbol (1)
timesteps = 300 # timesteps
num_hidden = 128 # hidden layer num of features

In [6]:
def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
       
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list

In [7]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [8]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [9]:
#prediction = tf.nn.softmax(logits)
inputs = tf.placeholder(tf.float32, [None, None, num_features])
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])




logits = RNN(inputs, weights, biases)

loss =  tf.nn.ctc_loss ( targets, logits , seq_len , time_major = False)
cost = tf.reduce_mean(loss)
optimizer = tf.train.MomentumOptimizer(learning_rate,
                                           0.9).minimize(cost)


# Option 2: tf.contrib.ctc.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
decoder_input = tf.transpose(logits, [1, 0, 2])

decoded, log_prob = tf.nn.ctc_greedy_decoder(decoder_input, seq_len)


In [10]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [11]:
init = tf.global_variables_initializer()

In [12]:
# need this for decoding word give its index key value 

def keys_of_value(dct, value):
    for k in dct:
        if isinstance(dct[k], list):
            if value in dct[k]:
                return k
        else:
            if value == dct[k]:
                return k

In [13]:
with tf.Session() as sess:
    

    # Run the initializer
                sess.run(init)
        
                 # Restore model weights from previously saved model
                load_path = saver.restore(sess, model_path)
                print("Model restored from file: %s" % save_path)
    
                
                max_len=timesteps
                    
                                 
                temp_np_inputs = np.zeros((batch_size , max_len , 13 ))
                    
                test_item = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')
                
                batch_lengths=[test_item.shape[0]]
                
                if test_item.shape[0] < max_len:
    
                        difference = max_len - test_item.shape[0]
    
                        temp_np_inputs[0] = np.pad(test_item,((0,difference),(0,0)), mode="constant")
        
           

                
            
                feed = {inputs: temp_np_inputs,
                        seq_len: batch_lengths}
	    

            
                
               
                
                # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
                str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in np.asarray(d[1]) ])
    
    
                print(str_decoded)  
 
    
    
    
    

INFO:tensorflow:Restoring parameters from /home/saurabh/Documents/tf_orange/tf_orange/models/model.ckpt


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [1,256,9] rhs shape= [2,256,9]
	 [[Node: save/Assign_1 = Assign[T=DT_FLOAT, _class=["loc:@Variable"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable/Momentum, save/RestoreV2_1)]]

Caused by op 'save/Assign_1', defined at:
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "